In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
#from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.callbacks import EarlyStopping

2023-05-15 10:46:32.949487: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("ADAYN_train_augmented.csv")
print(df.shape)
df.head()

(1000, 51530)


,0,1,2,3,4,5,6,7,8,9,...,51520,51521,51522,51523,51524,51525,51526,51527,51528,label
0,0.254902,0.250980,0.247059,0.247059,0.247059,0.247059,0.250980,0.254902,0.262745,0.274510,...,0.227451,0.223529,0.223529,0.223529,0.223529,0.223529,0.223529,0.223529,0.219608,2.0
1,0.592157,0.592157,0.592157,0.596078,0.600000,0.600000,0.596078,0.592157,0.596078,0.592157,...,0.603922,0.611765,0.619608,0.623529,0.623529,0.627451,0.627451,0.619608,0.615686,0.0
2,0.572549,0.564706,0.552941,0.537255,0.517647,0.501961,0.494118,0.490196,0.509804,0.533333,...,0.556863,0.568627,0.568627,0.564706,0.572549,0.580392,0.568627,0.545098,0.529412,4.0
3,0.670588,0.662745,0.635294,0.627451,0.682353,0.678431,0.631373,0.654902,0.560784,0.494118,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
4,0.541176,0.572549,0.556863,0.552941,0.631373,0.635294,0.596078,0.576471,0.584314,0.635294,...,0.584314,0.596078,0.576471,0.521569,0.509804,0.568627,0.611765,0.623529,0.580392,0.0


In [3]:
y_train = np.array(df["label"])
X_train = np.array(df.drop(columns=["label"])).reshape(-1, 227, 227)
print(y_train.shape)
print(X_train.shape)

(1000,)
(1000, 227, 227)


# 7. Apply Networks

# 7.1 AlexNet

clear

In [4]:
keras.backend.clear_session() # clear any previous models

build model

In [4]:
model = keras.Sequential()
    
model.add(keras.layers.Conv2D(96, (11, 11), strides=(4, 4), padding='valid', activation='relu', input_shape=(227, 227, 1))) # layer that convolves the input image with 96 different filters that are 11x11 pixels in size
model.add(keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')) # layer that downsamples the input representation by taking the maximum value over the window defined by pool_size for each dimension along the features axis
model.add(keras.layers.Conv2D(256, (5, 5), strides=(1, 1), padding='same', activation='relu')) # layer that convolves the input image with 256 different filters that are 5x5 pixels in size
model.add(keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')) # layer that downsamples the input representation by taking the maximum value over the window defined by pool_size for each dimension along the features axis
model.add(keras.layers.Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu')) # layer that convolves the input image with 384 different filters that are 3x3 pixels in size
model.add(keras.layers.Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu')) # layer that convolves the input image with 384 different filters that are 3x3 pixels in size
model.add(keras.layers.Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu')) # layer that convolves the input image with 384 different filters that are 3x3 pixels in size
model.add(keras.layers.Dense(4096, activation='relu')) # layer with 4096 neurons that uses the ReLU activation function and is fully connected
model.add(keras.layers.Flatten()) # layer that flattens the input
model.add(keras.layers.Dropout(0.5)) # layer that randomly sets input units to 0 with a 50% frequency at each step during training time, which helps prevent overfitting
model.add(keras.layers.Dense(4096, activation='relu')) # layer with 4096 neurons that uses the ReLU activation function and is fully connected
model.add(keras.layers.Dropout(0.5)) # layer that randomly sets input units to 0 with a 50% frequency at each step during training time, which helps prevent overfitting
model.add(keras.layers.Dense(8, activation='softmax')) # layer with 8 neurons that uses the softmax activation function to output probabilities for each class

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        11712     
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 96)        1115232   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5, 5, 96)         0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 5, 256)         614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2, 2, 256)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 2, 2, 384)         8

In [6]:

model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/10
25/25 [==============================] - 36s 1s/step - loss: 2.0544 - accuracy: 0.3113 - val_loss: 2.0394 - val_accuracy: 0.2450
Epoch 2/10
25/25 [==============================] - 34s 1s/step - loss: 2.0042 - accuracy: 0.3425 - val_loss: 2.0031 - val_accuracy: 0.2450
Epoch 3/10
25/25 [==============================] - 27s 1s/step - loss: 1.9566 - accuracy: 0.3425 - val_loss: 1.9702 - val_accuracy: 0.2450
Epoch 4/10
25/25 [==============================] - 28s 1s/step - loss: 1.9109 - accuracy: 0.3425 - val_loss: 1.9401 - val_accuracy: 0.2450
Epoch 5/10
25/25 [==============================] - 29s 1s/step - loss: 1.8695 - accuracy: 0.3425 - val_loss: 1.9138 - val_accuracy: 0.2450
Epoch 6/10
25/25 [==============================] - 25s 1s/step - loss: 1.8285 - accuracy: 0.3425 - val_loss: 1.8920 - val_accuracy: 0.2450
Epoch 7/10
25/25 [==============================] - 27s 1s/step - loss: 1.7885 - accuracy: 0.3425 - val_loss: 1.8769 - val_accuracy: 0.2450
Epoch 8/10
25/25 [==

In [9]:
# Evaluate the model
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

# Sgow the test accuracy
print(test_acc)

NameError: name 'history' is not defined

In [10]:
y_pred_prob = model.predict(X_train)
y_pred_class = y_pred_prob.argmax(axis=-1)
print(X_train.shape)
print(y_pred_prob.shape)
print(y_pred_class.shape)

32/32 [==============================] - 7s 190ms/step
(1000, 227, 227)
(1000, 8)
(1000,)


In [17]:
unique, counts = np.unique(y_train, return_counts=True)
print(unique, counts)

[0. 1. 2. 3. 4. 5. 6. 7.] [323 277  58  89  91   6 102  54]


# sklearn random serach function for a neural network
param_grid = {"optimizer": ["adam", "sgd", "rmsprop"],
              "learning_rate": [0.001, 0.01, 0.1],
              "momentum": [0.0, 0.2, 0.4, 0.6, 0.8, 0.9],
              "epochs": [10, 20, 30, 40, 50],
              "batch_size": [32, 64, 128, 256],
            "loss": ["categorical_crossentropy", "sparse_categorical_crossentropy"],

            
              #"early_stopping": = EarlyStopping(monitor='val_loss', patience=2)
              # "activation": ["relu", "sigmoid", "tanh"],
              # "dropout_rate": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
              # "weight_constraint": [1, 2, 3, 4, 5],
              # "neurons": [1, 5, 10, 15, 20, 25, 30]
              }

def random_search(X_train, y_train, X_test, y_test, param_grid, n_iter_search=20, n_jobs=-1, cv=3, verbose=1, random_state=1): 
    # create a random search object
    random_search = RandomizedSearchCV(estimator=model_alex,
                                        param_distributions=param_grid,
                                        n_iter=n_iter_search,
                                        n_jobs=n_jobs,
                                        cv=cv,
                                        verbose=0,
                                        random_state=random_state)
    # fit the random search model
    random_search.fit(X_train, y_train)
    # print results
    print("Best Score: ", random_search.best_score_)
    print("Best Params: ", random_search.best_params_)
    # evaluate the model on test data
    print("Test Accuracy: ", random_search.score(X_test, y_test))
    return random_search

random_search(X_t_A_A, y_t_A_A, X_test, y_test, param_grid)